In [1]:
%reload_ext autoreload
%autoreload 2
import dpu_clf
import gym
import dotenv
dotenv.load_dotenv()

c:\Users\matan\anaconda3\envs\procgen_env_clone\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [26]:
seed = 8798

env_kwargs = {
    "render_mode": "rgb_array",
    "num_levels": 0,
    "start_level": 0,
    "distribution_mode": "easy",
    "rand_seed": seed,
    "use_discrete_action_wrapper": True, 
    "use_stay_bonus_wrapper": False,
    'food_diversity': 4,
    'fruitbot_num_walls': 5,
    'fruitbot_num_good_min': 5,
    'fruitbot_num_good_range': 1,
    'fruitbot_num_bad_min': 5,
    'fruitbot_num_bad_range': 1,
    'fruitbot_wall_gap_pct': 50,
    'fruitbot_door_prob_pct': 0,
    "fruitbot_reward_positive": 2,
    "fruitbot_reward_negative": -1,
    "fruitbot_reward_wall_hit": -2,
    }


env = gym.make("procgen:procgen-fruitbot-v0", **env_kwargs)

# model_path = "models\\fruitbot\\20251214-084655_easy\\ppo_final.zip"
# model_path2 = "models\\fruitbot\\20251213-212435_easy\\ppo_final.zip"
# agent = dpu_clf.load_agent(None, model_path2)

starting make_env with env_name: fruitbot
Using prebuilt binaries from: c:\Users\matan\master_thesis\rl_envs\procgen\procgen\.build\relwithdebinfo\RelWithDebInfo
ExtractDictObWrapper applied


In [14]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from stable_baselines3 import PPO
from IPython.display import display
from PIL import Image

## Detailed Episode Data

You can access detailed per-episode data for each model:

In [33]:
# Find all models in the fruitbot folder
models_base_dir = Path("models/fruitbot")
model_dirs = sorted([d for d in models_base_dir.iterdir() if d.is_dir()])

print(f"Found {len(model_dirs)} model directories:")
for model_dir in model_dirs:
    model_file = model_dir / "ppo_final.zip"
    exists = "✓" if model_file.exists() else "✗"
    print(f"  {exists} {model_dir.name}")

Found 21 model directories:
  ✓ 20251118-115646_easy
  ✓ 20251118-164559_easy
  ✓ 20251118-191507_easy
  ✓ 20251119-112830_easy
  ✓ 20251126-215809_easy
  ✓ 20251127-165318_easy
  ✓ 20251130-001800_easy
  ✓ 20251130-093847_easy
  ✓ 20251201-002723_easy
  ✓ 20251201-110009_easy
  ✓ 20251201-191008_easy
  ✓ 20251203-104254_easy
  ✓ 20251203-132922_easy
  ✓ 20251213-212435_easy
  ✓ 20251214-084655_easy
  ✓ 20251214-120912_easy
  ✓ 20251215-094437_easy
  ✓ 20251215-173855_easy
  ✓ 20251216-164146_easy
  ✓ 20251216-200621_easy
  ✓ 20251217-003433_easy


In [30]:
def evaluate_model(model_path, env, num_episodes=10, 
                   fruitbot_reward_positive=2.0,
                   fruitbot_reward_negative=-1.0,
                   fruitbot_reward_wall_hit=-2.0):
    """
    Evaluate a single model and return episode statistics.;
    
    Returns:
        dict with per-episode data and summary statistics
    """
    # Load model
    model = PPO.load(model_path)
    
    # Track per-episode data
    episodes_data = []
    
    for ep in range(num_episodes):
        obs = env.reset()
        if isinstance(obs, tuple):
            obs, _ = obs
        
        done = False
        truncated = False
        total_reward = 0.0
        steps = 0
        good_food = 0
        bad_food = 0
        wall_hits = 0
        first_frame = True
        while not (done or truncated):
            action, _states = model.predict(obs, deterministic=True)
            result = env.step(action)
            
            # Handle both gym and gymnasium API
            if len(result) == 4:
                obs, reward, done, info = result
                truncated = False
            else:
                obs, reward, done, truncated, info = result

            # if first_frame:
            #     game_image = info.get('rgb', None)
            #     game_image = Image.fromarray(game_image).convert('RGB').resize((512, 512))
            #     display(game_image)
            #     first_frame = False

            
            # Ensure reward is a scalar float
            try:
                r = float(np.asarray(reward).item())
            except Exception:
                r = float(reward)
            
            total_reward += r
            steps += 1
            # Count events based on reward values
            TOL = 1e-3
            if np.isclose(r, fruitbot_reward_positive, atol=TOL, rtol=0.0):
                good_food += 1
            elif np.isclose(r, fruitbot_reward_negative, atol=TOL, rtol=0.0):
                bad_food += 1
            elif np.isclose(r, fruitbot_reward_wall_hit, atol=TOL, rtol=0.0):
                wall_hits += 1
        
        episodes_data.append({
            'episode': ep + 1,
            'reward': total_reward,
            'steps': steps,
            'good_food': good_food,
            'bad_food': bad_food,
            'wall_hits': wall_hits
        })
    
    # Calculate summary statistics
    rewards = [ep['reward'] for ep in episodes_data]
    lengths = [ep['steps'] for ep in episodes_data]
    good_foods = [ep['good_food'] for ep in episodes_data]
    bad_foods = [ep['bad_food'] for ep in episodes_data]
    wall_hits_list = [ep['wall_hits'] for ep in episodes_data]
    
    summary = {
        'mean_reward': np.mean(rewards),
        'std_reward': np.std(rewards),
        'mean_steps': np.mean(lengths),
        'mean_good_food': np.mean(good_foods),
        'mean_bad_food': np.mean(bad_foods),
        'mean_wall_hits': np.mean(wall_hits_list),
        'episodes_data': episodes_data
    }
    
    return summary

In [32]:
#create the environment
seed = 8798
# Use the reward values from your env_kwargs or set them explicitly
fruitbot_reward_positive = 2.0
fruitbot_reward_negative = -1.0
fruitbot_reward_wall_hit = -2.0

env_kwargs = {
    "render_mode": "rgb_array",
    "num_levels": 0,
    "start_level": 0,
    "distribution_mode": "easy",
    "rand_seed": seed,
    "use_discrete_action_wrapper": True, 
    "use_stay_bonus_wrapper": False,
    'food_diversity': 4,
    'fruitbot_num_walls': 5,
    'fruitbot_num_good_min': 5,
    'fruitbot_num_good_range': 1,
    'fruitbot_num_bad_min': 5,
    'fruitbot_num_bad_range': 1,
    'fruitbot_wall_gap_pct': 50,
    'fruitbot_door_prob_pct': 0,
    "fruitbot_reward_positive": fruitbot_reward_positive,
    "fruitbot_reward_negative": fruitbot_reward_negative,
    "fruitbot_reward_wall_hit": fruitbot_reward_wall_hit,
    }


env = gym.make("procgen:procgen-fruitbot-v0", **env_kwargs)


# Evaluate all models
results = []



for i, model_dir in enumerate(model_dirs):
    model_file = model_dir / "ppo_final.zip"
    
    if not model_file.exists():
        print(f"Skipping {model_dir.name} - model file not found")
        continue
    
    try:
        print(f"\n[{i+1}/{len(model_dirs)}] Evaluating {model_dir.name}...")
        
        # Evaluate model
        eval_results = evaluate_model(
            model_path=str(model_file).replace('.zip', ''),  # PPO.load adds .zip automatically
            env=env,
            num_episodes=30,
            fruitbot_reward_positive=fruitbot_reward_positive,
            fruitbot_reward_negative=fruitbot_reward_negative,
            fruitbot_reward_wall_hit=fruitbot_reward_wall_hit
        )
        
        # Store summary results
        results.append({
            'model_name': model_dir.name,
            'mean_reward': eval_results['mean_reward'],
            'std_reward': eval_results['std_reward'],
            'mean_steps': eval_results['mean_steps'],
            'mean_good_food': eval_results['mean_good_food'],
            'mean_bad_food': eval_results['mean_bad_food'],
            'mean_wall_hits': eval_results['mean_wall_hits'],
            'episodes_data': eval_results['episodes_data']
        })
        
        print(f"  Mean reward: {eval_results['mean_reward']:.2f} ± {eval_results['std_reward']:.2f}")
        print(f"  Mean steps: {eval_results['mean_steps']:.1f}")
        print(f"  Mean good food: {eval_results['mean_good_food']:.2f}")
        print(f"  Mean bad food: {eval_results['mean_bad_food']:.2f}")
        print(f"  Mean wall hits: {eval_results['mean_wall_hits']:.2f}")
        
    except Exception as e:
        print(f"Error evaluating {model_dir.name}: {e}")
        continue

print(f"\n{'='*60}")
print(f"Evaluation complete! Evaluated {len(results)} models.")
print(f"{'='*60}")

starting make_env with env_name: fruitbot
Using prebuilt binaries from: c:\Users\matan\master_thesis\rl_envs\procgen\procgen\.build\relwithdebinfo\RelWithDebInfo
ExtractDictObWrapper applied

[1/21] Evaluating 20251118-115646_easy...
Error evaluating 20251118-115646_easy: No module named 'gymnasium'

[2/21] Evaluating 20251118-164559_easy...
Error evaluating 20251118-164559_easy: No module named 'gymnasium'

[3/21] Evaluating 20251118-191507_easy...
Error evaluating 20251118-191507_easy: No module named 'gymnasium'

[4/21] Evaluating 20251119-112830_easy...
Error evaluating 20251119-112830_easy: No module named 'gymnasium'

[5/21] Evaluating 20251126-135043_easy...
  Mean reward: 7.70 ± 11.28
  Mean steps: 85.1
  Mean good food: 1.70
  Mean bad food: 1.20
  Mean wall hits: 0.60

[6/21] Evaluating 20251126-191339_easy...
  Mean reward: 7.70 ± 11.28
  Mean steps: 85.1
  Mean good food: 1.70
  Mean bad food: 1.20
  Mean wall hits: 0.60

[6/21] Evaluating 20251126-191339_easy...
  Mean rew

In [ ]:
# Create summary DataFrame
summary_df = pd.DataFrame([
    {
        'model_name': r['model_name'],
        'mean_reward': r['mean_reward'],
        'std_reward': r['std_reward'],
        'mean_steps': r['mean_steps'],
        'mean_good_food': r['mean_good_food'],
        'mean_bad_food': r['mean_bad_food'],
        'mean_wall_hits': r['mean_wall_hits']
    }
    for r in results
])

# Sort by mean reward (descending)
summary_df = summary_df.sort_values('mean_reward', ascending=False)

print("Summary of all models (sorted by mean reward):")
print(summary_df.to_string(index=False))

# Save to CSV
summary_df.to_csv('model_evaluation_summary.csv', index=False)
print("\nResults saved to 'model_evaluation_summary.csv'")

In [ ]:
# Example: View detailed episodes for the best model
if results:
    best_model = summary_df.iloc[0]['model_name']
    best_model_data = next(r for r in results if r['model_name'] == best_model)
    
    print(f"Detailed episodes for best model: {best_model}\n")
    episodes_df = pd.DataFrame(best_model_data['episodes_data'])
    print(episodes_df.head(10))
    
    # Save detailed data for all models
    all_episodes = []
    for r in results:
        for ep_data in r['episodes_data']:
            all_episodes.append({
                'model_name': r['model_name'],
                **ep_data
            })
    
    all_episodes_df = pd.DataFrame(all_episodes)
    all_episodes_df.to_csv('model_evaluation_detailed.csv', index=False)
    print(f"\nDetailed episode data saved to 'model_evaluation_detailed.csv'")
    print(f"Total episodes: {len(all_episodes_df)}")